## 파이토치 심화

### 배치정규화 
내부 공변량 변화를 줄이기 위해 사용함. 4가지 정규화 방식이 있음
- 배치 정규화 : CNN, MLP 같은 순방향 신경망에 쓰임
- 계층 정규화 : RNN, 트랜스포머에 쓰임
- 인스턴스 정규화 : 스타일 변환, GAN에 쓰임
- 그룹정규화 : CNN 배치 사이즈가 작을 때

In [2]:
# 정규화 예시
import torch
from torch import nn
x = torch.FloatTensor(
    [
        [-0.65,-0.57,0.63],
        [0.73,0.21,1.52],
        [0.24,0.56,0.32]
    ]
)

print("정규화 안했을 때 :", x)
print("정규화 했을 때 :", nn.BatchNorm1d(3)(x))

정규화 안했을 때 : tensor([[-0.6500, -0.5700,  0.6300],
        [ 0.7300,  0.2100,  1.5200],
        [ 0.2400,  0.5600,  0.3200]])
정규화 했을 때 : tensor([[-1.3246, -1.3479, -0.3801],
        [ 1.0912,  0.3035,  1.3697],
        [ 0.2334,  1.0445, -0.9896]], grad_fn=<NativeBatchNormBackward0>)


## 가중치 초기화
모델의 초기 가중치를 설정하는 것
gradient vanishing, gradient exposive를 방지 가능
- 상수 초기화
- 무작위 초기화
- 제르비어 초기화 : 활성화 함수가 시그모이드 형식일때 좋음
- 카이밍(허) 초기화 : ReLU에서 죽은 뉴런 문제 해결 가능, 활성화 함수가 ReLU일때 좋음
- 직교 초기화 : 특이값분해(SVD)를 이용함, LSTM, GRU같은 RNN에 주로 사용

In [3]:
# 제르비어 초기화 예시 1

from torch import nn

class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1,2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2,1)

    def _init_weights(self):
        nn.init.xavier_uniform_(self.layer[0].weight)
        self.layer[0].bias.data.fiil_(0.01)
        
        nn.init.xavier_uniform_(self.fc.weight)
        self.fc.bias.data.fiil_(0.01)

In [5]:
# 복잡한 모델에서는 모듈화해서 가중치를 초기화함

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1,2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2,1)
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):    
            nn.init.xavier_uniform_(module.weight)
            nn.init.constant_(module.bias,0.01)
        print(f"apply {module}")
        
model = Net()

apply Linear(in_features=1, out_features=2, bias=True)
apply Sigmoid()
apply Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
apply Linear(in_features=2, out_features=1, bias=True)
apply Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)


### 정칙화
L1 정칙화 Lasso 
L2 정칙화 Ridge 
가 있음
릿지는 optimizer에서 weight_decay를 통해서도 구현가능